In [1]:
pip install QuantLib-Python


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\users\jyb\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


## 옵션의 가격을 계산하는 예시

### 미국형옵션

In [ ]:
# 아메리칸 스타일의 콜 옵션 가격을 계산해 보겠습니다. 이 예제에서는 Binomial Tree 모델을 사용

In [ ]:
# 아메리칸 스타일 콜 옵션의 가격을 계산하기 위해 Binomial Tree 모델(Cox-Ross-Rubinstein 모델)을 사용합니다.
# BlackScholesMertonProcess를 설정하고, 이를 바탕으로 BinomialVanillaEngine을 사용하여 옵션의 가격을 계산합니다. 
# 아메리칸 옵션은 만기 전 언제든지 실행될 수 있기 때문에, 유럽식 옵션보다 가치가 더 높거나 같습니다.

In [9]:
import QuantLib as ql

# 기본 설정
valuation_date = ql.Date(15, 3, 2022)
ql.Settings.instance().evaluationDate = valuation_date

# 옵션 파라미터
expiration_date = ql.Date(15, 9, 2022)
underlying_price = 100.0  # 기초 자산 가격
strike_price = 100.0  # 행사 가격
volatility = 0.20  # 변동성 (20%)
dividend_rate = 0.0  # 배당률
option_type = ql.Option.Call  # 콜 옵션

# 이자율 및 배당률 설정
risk_free_rate = 0.01  # 무위험 이자율 (1%)
day_count = ql.Actual365Fixed()
calendar = ql.UnitedStates()

# 이자율 곡선 구성
yield_curve = ql.FlatForward(valuation_date, risk_free_rate, day_count)
yield_curve_handle = ql.YieldTermStructureHandle(yield_curve)

# 배당률 곡선 구성
dividend_yield = ql.FlatForward(valuation_date, dividend_rate, day_count)
dividend_yield_handle = ql.YieldTermStructureHandle(dividend_yield)

# 변동성 곡선 구성
volatility_curve = ql.BlackConstantVol(valuation_date, calendar, volatility, day_count)
volatility_curve_handle = ql.BlackVolTermStructureHandle(volatility_curve)

# 아메리칸 옵션 정의
payoff = ql.PlainVanillaPayoff(option_type, strike_price)
settlement = valuation_date
am_exercise = ql.AmericanExercise(settlement, expiration_date)
american_option = ql.VanillaOption(payoff, am_exercise)

# 블랙-숄즈 프로세스 설정
bs_process = ql.BlackScholesMertonProcess(ql.QuoteHandle(ql.SimpleQuote(underlying_price)),
                                          dividend_yield_handle,
                                          yield_curve_handle,
                                          volatility_curve_handle)

# Binomial Tree 모델을 사용한 옵션 가격 책정
steps = 200
binomial_engine = ql.BinomialVanillaEngine(bs_process, "crr", steps)
american_option.setPricingEngine(binomial_engine)

# 옵션 가격 계산
option_price = american_option.NPV()

print(f"American Call Option Price: {option_price:.2f}")


American Call Option Price: 5.89


### 유럽형옵션 

In [ ]:
# 유럽식 옵션은 만기일에만 실행할 수 있으므로, EuropeanExercise를 사용하여 옵션의 실행 유형을 정의합니다. 
# 또한, 유럽식 옵션 가격 계산에는 일반적으로 AnalyticEuropeanEngine을 사용할 수 있습니다. 
# 이는 블랙-숄즈 모델을 기반으로 한 분석적 해법을 제공합니다.

In [ ]:
#  유럽식 콜 옵션의 가격을 계산합니다. EuropeanExercise를 사용하여 옵션의 실행 유형을 정의하고, 
#     AnalyticEuropeanEngine을 사용하여 옵션의 가격을 계산합니다. 
#     이 엔진은 블랙-숄즈 모델을 기반으로 한 분석적 해법을 제공하여, 유럽식 옵션의 가격을 효율적으로 계산할 수 있게 합니다.

In [10]:
import QuantLib as ql

# 기본 설정
valuation_date = ql.Date(15, 3, 2022)
ql.Settings.instance().evaluationDate = valuation_date

# 옵션 파라미터
expiration_date = ql.Date(15, 9, 2022)
underlying_price = 100.0  # 기초 자산 가격
strike_price = 100.0  # 행사 가격
volatility = 0.20  # 변동성 (20%)
dividend_rate = 0.0  # 배당률
option_type = ql.Option.Call  # 콜 옵션

# 이자율 및 배당률 설정
risk_free_rate = 0.01  # 무위험 이자율 (1%)
day_count = ql.Actual365Fixed()
calendar = ql.UnitedStates()

# 이자율 곡선 구성
yield_curve = ql.FlatForward(valuation_date, risk_free_rate, day_count)
yield_curve_handle = ql.YieldTermStructureHandle(yield_curve)

# 배당률 곡선 구성
dividend_yield = ql.FlatForward(valuation_date, dividend_rate, day_count)
dividend_yield_handle = ql.YieldTermStructureHandle(dividend_yield)

# 변동성 곡선 구성
volatility_curve = ql.BlackConstantVol(valuation_date, calendar, volatility, day_count)
volatility_curve_handle = ql.BlackVolTermStructureHandle(volatility_curve)

# 유럽식 옵션 정의
payoff = ql.PlainVanillaPayoff(option_type, strike_price)
eu_exercise = ql.EuropeanExercise(expiration_date)
european_option = ql.VanillaOption(payoff, eu_exercise)

# 블랙-숄즈 프로세스 설정
bs_process = ql.BlackScholesMertonProcess(ql.QuoteHandle(ql.SimpleQuote(underlying_price)),
                                          dividend_yield_handle,
                                          yield_curve_handle,
                                          volatility_curve_handle)

# 분석적 유럽식 엔진을 사용한 옵션 가격 책정
european_option.setPricingEngine(ql.AnalyticEuropeanEngine(bs_process))

# 옵션 가격 계산
option_price = european_option.NPV()

print(f"European Call Option Price: {option_price:.2f}")


European Call Option Price: 5.90


## 채권의 내재 수익률(Yield to Maturity, YTM)

In [ ]:
# 내재 수익률은 채권의 현재 가격이 미래의 모든 현금 흐름(쿠폰 지급 및 만기 시 원금 상환)의 현재 가치와 같게 만드는 할인율입니다. 
# 이는 투자자가 채권을 현재 가격으로 구매하여 만기까지 보유했을 때 기대할 수 있는 수익률을 나타냅니다.

In [7]:
import QuantLib as ql

# 기본 설정
valuation_date = ql.Date(15, 3, 2022)
ql.Settings.instance().evaluationDate = valuation_date

# 채권 파라미터
issue_date = ql.Date(15, 3, 2020)
maturity_date = ql.Date(15, 3, 2025)
tenor = ql.Period(ql.Annual)
calendar = ql.UnitedStates()
business_convention = ql.Unadjusted
date_generation = ql.DateGeneration.Backward
month_end = False
coupon_rate = 0.05  # 5%
coupons = [coupon_rate]
day_count = ql.Thirty360()
face_value = 100

# 채권 스케줄 생성
schedule = ql.Schedule(issue_date, maturity_date, tenor, calendar, business_convention,
                       business_convention, date_generation, month_end)

# 고정금리 채권 생성 (settlementDays 인자 제거)
fixed_rate_bond = ql.FixedRateBond(3, face_value, schedule, coupons, day_count)

# 채권 현재 가격 (시장 가격)
market_price = 95.0  # 예: 시장에서 채권의 현재 가격이 95

# 내재 수익률 계산
ytm = fixed_rate_bond.bondYield(market_price, day_count, ql.Compounded, ql.Annual)

print(f"채권의 내재 수익률(YTM): {ytm:.2%}")


채권의 내재 수익률(YTM): 6.89%


## 이자율 스왑 계약의 현재 가치를 계산


In [ ]:
# 고정 금리와 변동 금리를 교환하는 이자율 스왑의 순현재가치(NPV)를 계산
# 여기서는 간단히 고정 금리와 변동 금리(USD LIBOR)에 대한 가정을 바탕으로 스왑 계약을 설정하고, QuantLib의 기능을 사용하여 NPV를 계산합니다. 
# 실제 환경에서는 시장 데이터를 바탕으로 보다 정교한 이자율 곡선을 구성해야 합니다.

In [8]:
import QuantLib as ql

# 날짜 설정
valuation_date = ql.Date(15, 3, 2022)
ql.Settings.instance().evaluationDate = valuation_date

# 이자율 곡선 구성
calendar = ql.UnitedStates()
bussiness_convention = ql.Unadjusted
day_count = ql.Thirty360()
end_of_month = False
settlement_days = 2
face_amount = 1000000  # 100만 달러
coupon_rate = 0.025  # 고정 금리 2.5%
coupons = [coupon_rate]

# USD LIBOR 곡선
libor_rate = 0.02  # 부동 금리 가정
libor_day_count = ql.Actual360()
forward_rate = ql.SimpleQuote(libor_rate)
libor_curve = ql.FlatForward(valuation_date, ql.QuoteHandle(forward_rate), libor_day_count)
libor_curve_handle = ql.YieldTermStructureHandle(libor_curve)

# 고정 금리 스왑 스케줄
start_date = ql.Date(20, 3, 2022)
end_date = ql.Date(15, 3, 2027)
fixed_schedule = ql.Schedule(start_date, end_date, ql.Period(ql.Annual), calendar,
                             bussiness_convention, bussiness_convention, ql.DateGeneration.Forward, end_of_month)

# 부동 금리 스왑 스케줄
float_schedule = ql.Schedule(start_date, end_date, ql.Period(ql.Quarterly), calendar,
                             bussiness_convention, bussiness_convention, ql.DateGeneration.Forward, end_of_month)

# 스왑 정의
fixed_leg = ql.FixedRateLeg(fixed_schedule, day_count, [face_amount], coupons)
libor_index = ql.USDLibor(ql.Period(3, ql.Months), libor_curve_handle)
floating_leg = ql.IborLeg([face_amount], float_schedule, libor_index)

# 스왑 가치 평가
swap = ql.Swap(fixed_leg, floating_leg)
swap.setPricingEngine(ql.DiscountingSwapEngine(libor_curve_handle))

# 결과 출력
npv = swap.NPV()
print(f"Swap NPV: {npv:.2f}")


Swap NPV: -21137.18


## 유럽식 콜 옵션의 델타 헤징 예시

In [ ]:
# 델타 헤징은 옵션 포지션의 가격 변동 위험을 관리하기 위해 기초 자산(예: 주식)을 매매함으로써 옵션의 델타를 중립화하는 전략입니다. 
# 델타는 옵션 가격이 기초 자산 가격 변동에 대해 얼마나 민감한지를 나타내는 척도

In [ ]:
# 델타 헤징 전략에서는 이 델타 값을 사용하여 옵션 포지션의 가격 변동 위험을 중립화합니다. 
# 구체적으로, 옵션 포지션의 델타와 반대 방향으로 기초 자산을 매수하거나 매도함으로써 전체 포지션의 델타를 0에 가깝게 조정합니다. 
# 이를 통해 기초 자산 가격의 변동으로 인한 옵션 포지션의 가치 변동을 최소화할 수 있습니다.
# 계산된 델타 값과 헤지 비율을 바탕으로, 만약 옵션 포지션의 델타가 양수라면 기초 자산을 해당 델타 값만큼 매도하여 전체 포지션의 델타를 0에 가깝게 만들 수 있습니다. 
# 반대로, 옵션 포지션의 델타가 음수라면 기초 자산을 매수하여 델타를 중립화할 수 있습니다.

In [12]:
import QuantLib as ql

# 기본 설정
valuation_date = ql.Date(15, 3, 2022)
ql.Settings.instance().evaluationDate = valuation_date

# 옵션 파라미터
expiration_date = ql.Date(15, 9, 2022)
underlying_price = 100.0  # 기초 자산 가격
strike_price = 100.0  # 행사 가격
volatility = 0.20  # 변동성 (20%)
dividend_rate = 0.0  # 배당률
option_type = ql.Option.Call  # 콜 옵션

# 이자율 및 배당률 설정
risk_free_rate = 0.01  # 무위험 이자율 (1%)
day_count = ql.Actual365Fixed()
calendar = ql.UnitedStates()

# 이자율 곡선 구성
yield_curve = ql.FlatForward(valuation_date, risk_free_rate, day_count)
yield_curve_handle = ql.YieldTermStructureHandle(yield_curve)

# 배당률 곡선 구성
dividend_yield = ql.FlatForward(valuation_date, dividend_rate, day_count)
dividend_yield_handle = ql.YieldTermStructureHandle(dividend_yield)

# 변동성 곡선 구성
volatility_curve = ql.BlackConstantVol(valuation_date, calendar, volatility, day_count)
volatility_curve_handle = ql.BlackVolTermStructureHandle(volatility_curve)

# 유럽식 옵션 정의
payoff = ql.PlainVanillaPayoff(option_type, strike_price)
eu_exercise = ql.EuropeanExercise(expiration_date)
european_option = ql.VanillaOption(payoff, eu_exercise)

# 블랙-숄즈 프로세스 설정
bs_process = ql.BlackScholesMertonProcess(ql.QuoteHandle(ql.SimpleQuote(underlying_price)),
                                          dividend_yield_handle,
                                          yield_curve_handle,
                                          volatility_curve_handle)

# 옵션 가격 책정 엔진 설정
european_option.setPricingEngine(ql.AnalyticEuropeanEngine(bs_process))

# 델타 계산
delta = european_option.delta()

# 델타 헤징을 위한 기초 자산 매수/매도 수량 계산
hedge_ratio = -delta  # 옵션 포지션과 반대 방향으로 기초 자산을 매수/매도해야 하므로 -delta를 사용
print(f"유럽식 콜 옵션의 델타: {delta:.4f}")
print(f"델타 헤징을 위해 필요한 기초 자산 매수/매도 수량: {hedge_ratio:.4f}")


유럽식 콜 옵션의 델타: 0.5424
델타 헤징을 위해 필요한 기초 자산 매수/매도 수량: -0.5424


In [ ]:
# 콜옵션 1개 매수 + 주식을 0.5424 매도
# 콜옵션 1개 매도 + 주식 0.5424주 매수

## 방어적 풋 전략(Protective Put Strategy)

In [ ]:
# 주식 포트폴리오의 하방 위험을 제한하기 위해 풋 옵션을 매수하는 전략 
# 이 전략은 주식의 소유자가 주식 가격이 하락할 경우의 손실을 제한하면서, 주식 가격 상승의 이익은 유지하고자 할 때 사용

In [ ]:
# 풋매수 필요 수량 계산 예시

In [13]:
import QuantLib as ql

# 기본 설정
valuation_date = ql.Date(15, 3, 2022)
ql.Settings.instance().evaluationDate = valuation_date

# 주식 및 옵션 파라미터
stock_price = 100.0  # 주식의 현재 가격
strike_price = 100.0  # 풋 옵션의 행사 가격
volatility = 0.20  # 변동성 (20%)
dividend_rate = 0.0  # 배당률
expiration_date = ql.Date(15, 9, 2022)  # 옵션 만기일

# 이자율 설정
risk_free_rate = 0.01  # 무위험 이자율 (1%)
day_count = ql.Actual365Fixed()
calendar = ql.UnitedStates()

# 이자율 곡선 구성
yield_curve = ql.FlatForward(valuation_date, risk_free_rate, day_count)
yield_curve_handle = ql.YieldTermStructureHandle(yield_curve)

# 배당률 곡선 구성
dividend_yield = ql.FlatForward(valuation_date, dividend_rate, day_count)
dividend_yield_handle = ql.YieldTermStructureHandle(dividend_yield)

# 변동성 곡선 구성
volatility_curve = ql.BlackConstantVol(valuation_date, calendar, volatility, day_count)
volatility_curve_handle = ql.BlackVolTermStructureHandle(volatility_curve)

# 블랙-숄즈 프로세스 설정
bs_process = ql.BlackScholesMertonProcess(ql.QuoteHandle(ql.SimpleQuote(stock_price)),
                                          dividend_yield_handle,
                                          yield_curve_handle,
                                          volatility_curve_handle)

# 풋 옵션 정의
payoff = ql.PlainVanillaPayoff(ql.Option.Put, strike_price)
exercise = ql.EuropeanExercise(expiration_date)
put_option = ql.VanillaOption(payoff, exercise)

# 옵션 가격 책정 엔진 설정
put_option.setPricingEngine(ql.AnalyticEuropeanEngine(bs_process))

# 풋 옵션 가격 계산
put_option_price = put_option.NPV()

print(f"풋 옵션 가격: {put_option_price:.2f}")


풋 옵션 가격: 5.40


In [ ]:
# 주식 포트폴리오의 총 가치가 $10,000이고, 현재 주식 가격이 $100일 때, 각 풋 옵션은 주식 1주를 커버합니다. 
# 따라서, 포트폴리오를 완전히 헤지하기 위해 필요한 풋 옵션의 수량은 포트폴리오 가치를 주식 가격으로 나눈 값과 같습니다.

In [14]:
# 주식 포트폴리오의 총 가치
portfolio_value = 10000.0  # 예: $10,000

# 현재 주식 가격
stock_price = 100.0  # 예: $100

# 필요한 풋 옵션의 수량 계산
put_options_needed = portfolio_value / stock_price

print(f"포트폴리오를 완전히 헤지하기 위해 필요한 풋 옵션의 수량: {put_options_needed:.0f}개")


포트폴리오를 완전히 헤지하기 위해 필요한 풋 옵션의 수량: 100개
